# Note on using MySQL with Python
### Creating a MySQL Database server on AWS EC2 

1. Create new instance (Using Ubuntu 18)
2. Allow MySQL Port 3306 in the security group
3. Fresh start setting up EC2 Machine
```
    3.0 export LC_ALL="en_US.UTF-8" to get rid of UTF error message
    
    3.1 sudo apt-get update && sudo apt-get upgrade
    
    3.2 Install MySQL https://dev.mysql.com/doc/mysql-apt-repo-quick-guide/en/#apt-repo-fresh-install
```
4. Start MySQL service
```
    4.0 Config binded port
        sudo vi /etc/mysql/my.cnf
        add 
            .
            .
            .
            [mysqld]
            bind-address=0.0.0.0
            .
            .
            .
    4.1 Restart MySQL service
        sudo services mysql restart
```

5. Add account

```
    mysql -u root -p 
    Enter Password: xxxxxxx
    
    > CREATE user 'tonny'@'%' IDENTIFIED BY '<password here>';
    > GRANT ALL PRIVILEGES ON *.* TO 'tonny'@'%' WITH GRANT OPTION;
    
```
Upload Scheme and good to go!!

```
    scp -i key source target
    mysql -u root -p < scheme.sql

```

In [2]:
## Creating Connection

import mysql.connector

cnx = mysql.connector.connect(user='jobdatabaseroot', password='jobdatabaseroot', host='jobdatabase2.cieq9tyvhuue.ap-southeast-1.rds.amazonaws.com', database='company_database')
cnx.close()

### NOTE : May try to use SQLAlchemy for ORM paradigm, but THIS doc use old school CRUD querying style.

# C : Create

In [5]:
cnx = mysql.connector.connect(user='tonny', password='tonny', host='13.250.107.59', database='job_database')
cursor = cnx.cursor()

query = '''
    CREATE TABLE `test`(
        `col1` INT NOT NULL AUTO_INCREMENT,
        `col2` VARCHAR(50),
        PRIMARY KEY (`col1`)
    ) ENGINE=InnoDB;
'''

cursor.execute(query)
cnx.close()

___
# R : Read

NOTE : Run the update process for a few times (with different PK) in order to see the data in the table.

In [17]:
cnx = mysql.connector.connect(user='tonny', password='tonny', host='13.250.107.59', database='job_database')
cursor = cnx.cursor()

query = '''
    SELECT * FROM test;
'''
cursor.execute(query)

for (item1, item2) in cursor:
    print("{} = {}".format(item1,item2))

cnx.close()

1 = TEST
2 = TEST


___
# U : Update

In [ ]:
cnx = mysql.connector.connect(user='tonny', password='tonny', host='13.250.107.59', database='job_database')
cursor = cnx.cursor()

query = '''
    INSERT INTO company(companyname, provinceid, industryid) VALUES(%s, %s, %s);
'''

query = '''
    INSERT INTO industry(industrynameEN, industrynameTH) VALUES(%s, %s);
'''

data = ("TEST INDUS", "อุตสาหกรรมทดสอบ")
cursor.execute(query, data)

cnx.commit()
cnx.close()

___
# D : Delete

In [23]:
## Do just like CRU but use DELETE instead

___
# Load .sql script



In [25]:
import os 

os.listdir("..")

['.DS_Store', 'schema.sql', 'notebook']

In [ ]:
## load script in a file
query=""
with open("../schema.sql", r, encoding='utf-8') as fin:
    for line in fin:
        query += line

cnx = mysql.connector.connect(user='tonny', password='tonny', host='13.250.107.59', database='job_database')
cursor = cnx.cursor()

cursor.execute(query)
cnx.commit()
cnx.close()

___
# MySQL Pandas

It is recommended that SQLAlchemy should be used as it provides ORM designing and integration with things.

In [6]:
# Reading into Pandas DataFrame

import pandas as pd

def getConnection():
    return mysql.connector.connect(user='tonny', password='tonny', host='13.250.107.59', database='job_database')

def readIntoDF(query, arguments=None):
    if(arguments != None):
        if (type(arguments) != tuple):
            raise TypeError("arguments is not of type tuple")
        cnx = getConnection()
        cursor = cnx.cursor()
        cursor.execute(query, arguments)
    else:
        cnx = getConnection()
        cursor = cnx.cursor()
        cursor.execute(query)
        
    header = cursor.column_names
    outData = pd.DataFrame(data=cursor.fetchall(), columns=header)
    cnx.close()
    return outData

def update(query, arguments=None):
    if(arguments != None):
        if (type(arguments) != tuple):
            raise TypeError("arguments is not of type tuple")
        cnx = getConnection()
        cursor = cnx.cursor()
        cursor.execute(query, arguments)
    else:
        cnx = getConnection()
        cursor = cnx.cursor()
        cursor.execute(query)
    cnx.commit()
    cnx.close()
    return True

In [20]:
readIntoDF("SELECT * FROM province")

,provinceid,provincenameEN,provincenameTH,regionid
0,1,Bangkok,กรุงเทพมหานคร,7
1,2,Samut Prakan,สมุทรปราการ,7
2,3,Nonthaburi,นนทบุรี,7
3,4,Pathum Thani,ปทุมธานี,7
4,5,Phra Nakhon Si Ayutthaya,พระนครศรีอยุธยา,2
5,6,Ang Thong,อ่างทอง,2
6,7,Loburi,ลพบุรี,2
7,8,Sing Buri,สิงห์บุรี,2
8,9,Chai Nat,ชัยนาท,2
9,10,Saraburi,สระบุรี,2


In [9]:
update("INSERT INTO industry(industrynameEN, industrynameTH) VALUES(%s, %s)", ("test industry", "อุตสาหกรรมทดสอบ"))

True